In [24]:
import torch
import numpy as np
from scenarionet import read_dataset_summary, read_scenario
from metadrive.engine.asset_loader import AssetLoader

In [76]:
class AdaptiveTokenizer:
    def __init__(self, importance_threshold=0.5, max_tokens=512, token_length=10):
        self.importance_threshold = importance_threshold
        self.max_tokens = max_tokens
        self.token_length = token_length
        self.type_importance = {
            'VEHICLE': 1.0,
            'PEDESTRIAN': 0.8,
            'BICYCLE': 0.7,
            'STATIC': 0.5,
            'UNKNOWN': 0.3
        }
        self.map_feature_importance = {
            'LANE': 0.9,
            'CROSSWALK': 0.7,
            'STOP_SIGN': 0.6,
            'TRAFFIC_LIGHT': 0.8,
            'UNKNOWN': 0.4
        }

    def score_importance(self, tracks, map_features):
        importance_map = np.zeros(len(tracks) + len(map_features))
        
        for i, (track_id, track) in enumerate(tracks.items()):
            state = track['state']
            valid = state['valid']
            if not valid.any():
                importance_map[i] = 0
                continue

            valid_positions = state['position'][valid]
            valid_velocities = state['velocity'][valid]
            valid_headings = state['heading'][valid]
            valid_lengths = state['length'][valid]
            valid_widths = state['width'][valid]
            valid_heights = state['height'][valid]
            
            activity_score = np.linalg.norm(valid_positions, axis=1).sum()
            velocity_score = np.linalg.norm(valid_velocities, axis=1).sum()
            heading_score = np.abs(valid_headings).sum()
            size_score = valid_lengths.sum() + valid_widths.sum() + valid_heights.sum()
            type_score = self.type_importance.get(track.get('type', 'UNKNOWN'), 0.3)
            
            importance_map[i] = activity_score + velocity_score + heading_score + size_score + type_score

        for i, (feature_id, feature) in enumerate(map_features.items(), start=len(tracks)):
            feature_type = feature.get('type', 'UNKNOWN')
            polyline = feature.get('polyline', [])
            feature_score = len(polyline) * self.map_feature_importance.get(feature_type, 0.4)
            importance_map[i] = feature_score
        
        importance_map = importance_map / (importance_map.max() + 1e-5)
        return importance_map

    def pad_or_truncate(self, token):
        if token.shape[0] > self.token_length:
            return token[:self.token_length]
        else:
            pad_length = self.token_length - token.shape[0]
            pad = np.zeros((pad_length, token.shape[1]), dtype=np.float32)
            return np.vstack((token, pad))

    def tokenize(self, tracks, map_features, metadata):
        importance_scores = self.score_importance(tracks, map_features)
        track_regions = []
        map_regions = []
        token_types = []
        token_ids = []

        for i, (track_id, track) in enumerate(tracks.items()):
            state = track['state']
            valid = state['valid']
            if not valid.any():
                continue
            
            valid_positions = state['position'][valid]
            valid_velocities = state['velocity'][valid]
            valid_headings = state['heading'][valid]
            valid_lengths = state['length'][valid]
            valid_widths = state['width'][valid]
            valid_heights = state['height'][valid]
            
            combined_token = np.column_stack((valid_positions, valid_velocities, valid_headings, valid_lengths, valid_widths, valid_heights))
            if importance_scores[i] >= self.importance_threshold:
                token = self.pad_or_truncate(combined_token)
                token_types.append(f'high-detail-{track.get("type", "UNKNOWN")}')
            else:
                token = self.pad_or_truncate(combined_token[::2])
                token_types.append(f'low-detail-{track.get("type", "UNKNOWN")}')
            
            track_regions.append(token)
            token_ids.append(track_id)

        for i, (feature_id, feature) in enumerate(map_features.items(), start=len(tracks)):
            polyline = feature.get('polyline', [])
            if len(polyline) > 0:
                polyline_array = np.array(polyline)
                token = self.pad_or_truncate(polyline_array)
                token_types.append(f'map-{feature.get("type", "UNKNOWN")}')
                map_regions.append(token)
                token_ids.append(feature_id)

        track_regions = track_regions[:self.max_tokens]
        map_regions = map_regions[:self.max_tokens]
        token_types = token_types[:self.max_tokens]
        token_ids = token_ids[:self.max_tokens]

        try:
            track_tensor = torch.tensor(np.stack(track_regions), dtype=torch.float32) if track_regions else torch.empty(0)
            map_tensor = torch.tensor(np.stack(map_regions), dtype=torch.float32) if map_regions else torch.empty(0)
        except ValueError as e:
            raise ValueError(f"Failed to stack token regions into tensors. Track shapes: {[t.shape for t in track_regions]}, Map shapes: {[m.shape for m in map_regions]}") from e

        return {
            'track_regions': track_tensor,
            'map_regions': map_tensor,
            'token_types': token_types,
            'token_ids': token_ids,
            'metadata': metadata
        }


In [78]:
sample_tracks = {
        'track_1': {'state': {'position': np.random.rand(10, 3), 'velocity': np.random.rand(10, 2), 'heading': np.random.rand(10), 
                              'length': np.random.rand(10), 'width': np.random.rand(10), 'height': np.random.rand(10), 
                              'valid': np.array([True] * 10)}, 'type': 'VEHICLE'}
    }
sample_map_features = {
        'lane_1': {'type': 'LANE', 'polyline': np.random.rand(5, 2)}
    }
sample_metadata = {'scenario_id': 'sample_001', 'map': 'city_map_1'}

tokenizer = AdaptiveTokenizer(token_length=10)
tokens = tokenizer.tokenize(sample_tracks, sample_map_features, sample_metadata)

print("Tokenization Complete!")
print(f"Track Tensor Shape: {tokens['track_regions'].shape}")
print(f"Map Tensor Shape: {tokens['map_regions'].shape}")
print(f"Token Types: {set(tokens['token_types'])}")
print(f"Token IDs: {tokens['token_ids']}")
print(f"Metadata: {tokens['metadata']}")

Tokenization Complete!


KeyError: 'token_regions'

In [73]:
av2_data =  AssetLoader.file_path("/home/light/Documents/Thesis/preprocessed_dataset", unix_style=False)
dataset_summary, scenario_ids, mapping = read_dataset_summary(dataset_path=av2_data)

scenario_file_name = scenario_ids[0]
scenario = read_scenario(dataset_path=av2_data, mapping=mapping, scenario_file_name=scenario_file_name)

In [74]:
scenario.keys()

dict_keys(['id', 'version', 'length', 'tracks', 'dynamic_map_states', 'map_features', 'metadata'])